# DataFrame

- 데이터프레임은 관계형 데이터베이스의 테이블에서 칼럼으로 구성된 변경 불가능한 분산 데이터 컬랙션이다.  
- 아파치 스파크 1.0에서 스키마 RDD가 소개됐고 스키마 RDD가 아파치 스파크 1.3버전에서 데이터프레임이라는 이름으로 바뀜.  
    - 스파크 1.x 버전에서는 데이터 프레임에 익숙하다면
    - 스파크 2.x 버전에서는 sQLContext 대신 SparkSession을 사용한다는걸 알수 있다
    - HiveContext, SQLContext, StreamingContext, SparkContext 등이 SparkSession으로 모두 통합되었다

- 파이썬 Pandas의 데이터프레임 또는 R의 데이터프레임에 익숙한 사람에게 스파크 데이터 프레임은 구조적인 데이터로 작업을 쉽게 해준다


## 파이썬 RDD의 약점
- 스파크 API를 파이썬에서 실행하는 것은 자바 JVM과 py4j 사이의 커뮤니케이션 오버헤드가 발생해서 느려질 수 있다.

In [5]:
import findspark, pyspark
findspark.find()

'C:\\Bigdata\\spark-2.4.5-bin-hadoop2.7'

In [10]:
# sc.stop()

In [11]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local[2]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [27]:
swimmersJSON = spark.read.json(
    "../../data/RDD_example/PySpark_data_key.txt/stringJSONRDD.json")
# 파일에 """ 이 포함되어 있어, 다음셀과 같이 진행

In [28]:
# 경로의 해당 파일 내용 복붙
stringJSONRDD = sc.parallelize(("""
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }
"""))

In [29]:
print(swimmersJSON)

DataFrame[_corrupt_record: string]


In [19]:
dir(swimmersJSON)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collectAsArrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_sort_cols',
 '_support_repr_html',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'exceptAll',
 'explain',
 'fillna',
 'filter',
 'first',
 'foreach',
 'f

In [35]:
swimmersJSON.createOrReplaceTempView('swimmersJSON')

In [36]:
swimmersJSON = spark.read.json(stringJSONRDD)

In [37]:
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [38]:
# createOrReplaceTempView 하고 나면 실행가능
spark.sql("SELECT * FROM swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

## RDD를 데이터프레임으로 변경하는 두가지 방법
- 리플렉션을 사용해 스키마를 추측하는 방법
- 스키마를 직접 코드상에 명시하는 방법

In [39]:
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [40]:
from pyspark.sql.types import *

In [41]:
stringCSVRDD = sc.parallelize([
      (123, 'Katie', 19, 'brown')
    , (234, 'Michael', 22, 'green')
    , (345, 'Simone', 23, 'blue')
])

schemaString = 'id name age eyeColor'

In [42]:
schema = StructType([
    StructField('id', LongType(), True)
    ,StructField('name', StringType(), True)
    ,StructField('age', LongType(), True)
    ,StructField('eyeColor', StringType(), True)
])

In [43]:
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [44]:
swimmers.createOrReplaceTempView('swimmers')

In [45]:
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



## Querying with SQL
- 다음은 DataBricks notebook 에서만 지원되는 방법
```sql
%sql
-- Query all data
select * from swimmers
```

In [47]:
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [48]:
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [50]:
spark.sql("select id, age from swimmers").show()

+---+---+
| id|age|
+---+---+
|123| 19|
|234| 22|
|345| 23|
+---+---+



In [55]:
spark.sql("SELECT id, age from swimmers where age=22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



## select


In [56]:
swimmers.select('id', "age").show()

+---+---+
| id|age|
+---+---+
|123| 19|
|234| 22|
|345| 23|
+---+---+



In [57]:
swimmers.select("id", "age").filter("age=22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [62]:
swimmers.select("name", "eyeColor").show()

+-------+--------+
|   name|eyeColor|
+-------+--------+
|  Katie|   brown|
|Michael|   green|
| Simone|    blue|
+-------+--------+



In [65]:
swimmers.select("name", "eyeColor").filter("eyeColor list 'b%' ").show()


ParseException: "\nextraneous input ''b%'' expecting <EOF>(line 1, pos 14)\n\n== SQL ==\neyeColor list 'b%' \n--------------^^^\n"

### spark SQL 과 데이터프레임으로 작업시 유의점
- CSV, JSON을 비롯한 다양한 데이터포멧을 작업하기 쉽지만 
- 가장 일반적인 포멧은 parquet파일 포맷이다. 
    - 스파크에서는 읽기 쓰기 다 지원하며 원본데이터의 스키마정보를 잘 보존해준다

In [90]:
# on-time flight Performance
# DataFrame Queries
flightPerfFilePath = "../../data/RDD_example/departuredelays.csv"
airportsFilePath = "../../data/RDD_example/airport-codes-na.txt"

In [91]:
airports = spark.read.csv( airportsFilePath
                          , header='true'
                          , inferSchema='true' # 리풀렉션으로 데이터 타입 추측 여부
                          , sep='\t' )

- spark.read.scv() 의 파라미터 인자 별 설명
    1. 데이터 파일경로
    1. 첫번째 줄이 헤더인지 여부
    1. 스파크가 리풀렉션으로 데이터 타입 추측 여부
    1. 구분자 지정

In [92]:
airports.createOrReplaceTempView('airports')

In [93]:
flightPerf = spark.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView('flightPerf')

In [94]:
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [101]:
# 다음과 같이 할 수도 잇음
spark.read.format("com.databricks.spark.csv")\
                .option("header", "true").load(airportsFilePath)

DataFrame[City	State	Country	IATA: string]

### 워싱턴에 위치해 있는 공항


In [102]:
# spark.sql('select * from airports where State = "WA"').show()
airports.select('*').filter('State = "WA"').show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Bellingham|   WA|    USA| BLI|
| Moses Lake|   WA|    USA| MWH|
|      Pasco|   WA|    USA| PSC|
|    Pullman|   WA|    USA| PUW|
|    Seattle|   WA|    USA| SEA|
|    Spokane|   WA|    USA| GEG|
|Walla Walla|   WA|    USA| ALW|
|  Wenatchee|   WA|    USA| EAT|
|     Yakima|   WA|    USA| YKM|
+-----------+-----+-------+----+



### 워싱턴에서 출발하는 비행기 중에서 지연이 된 경로 검색


In [98]:
spark.sql("SELECT a.City, f.origin, sum(f.delay) as Delays\
            FROM FlightPerf f\
            JOIN airports a on a.IATA = f.origin\
            WHERE a.State = 'WA'\
            GROUP BY a.City, f.origin\
            ORDER BY sum(f.delay) desc").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



```sql
SELECT a.City, f.origin, sum(f.delay) as Delays
FROM FlightPerf f
JOIN airports a on a.IATA = f.origin
WHERE a.State = 'WA'
GROUP BY a.City, f.origin
ORDER BY sum(f.delay) desc
```

### 미국의 주별 지연 정보 검색


In [99]:
spark.sql("SELECT a.State, sum(f.delay) as Delays\
            FROM FlightPerf f\
            JOIN airports a on a.IATA = f.origin\
            WHERE a.Country = 'USA'\
            GROUP BY a.State").show()

+-----+---------+
|State|   Delays|
+-----+---------+
|   SC|  80666.0|
|   AZ| 401793.0|
|   LA| 199136.0|
|   MN| 256811.0|
|   NJ| 452791.0|
|   OR| 109333.0|
|   VA|  98016.0|
| null| 397237.0|
|   RI|  30760.0|
|   WY|  15365.0|
|   KY|  61156.0|
|   NH|  20474.0|
|   MI| 366486.0|
|   NV| 474208.0|
|   WI| 152311.0|
|   ID|  22932.0|
|   CA|1891919.0|
|   CT|  54662.0|
|   NE|  59376.0|
|   MT|  19271.0|
+-----+---------+
only showing top 20 rows



### 유의 사항
- DataFrame은 스칼라나 자바에서 프로그래머가 정의한 사용자클래스에 의해  
타입을 확실히 명시해야 하는 JVM객체이다
    - 이는 타입에 대한 견고함이 적은 pySpark에서는 DataFrame API를 지원않는 경우가 잇다
        - PySpark에서 사용불가인 DataFrame API의 부분은 RDD로 변환 또는 UDF를 사용하면 가능해짐.

(http://bit.ly/2dbfoFT)